<a href="https://colab.research.google.com/github/ankitdhami29/Bike-Demand-Prediction-Model/blob/main/logo_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Apr 22 09:35:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install ultralytics

In [ ]:
!unzip -q /content/data.zip -d /content/custom_data

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# !python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

In [ ]:
# !pip install ultralytics

**# For training model**

In [ ]:
import os
import random
import shutil
import yaml
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from tqdm.notebook import tqdm

# Configuration
ORIG_IMAGE_DIR = Path("/content/content/content/images")
ORIG_LABEL_DIR = Path("/content/content/content/labels")
CLASSES_FILE = Path("/content/content/content/classes.txt")
YOLO_DATA_DIR = Path("/content/content/content/yolo_dataset")
TRAIN_RATIO = 0.8
MODEL_NAME = 'yolov8s.pt'
EPOCHS = 80
IMG_SIZE = 640
BATCH_SIZE = 8
INITIAL_LR = 0.01
FINAL_LR = 0.01

def find_image_path(image_dir, basename):
    supported_formats = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp')
    for ext in supported_formats:
        path_lower = image_dir / f"{basename}{ext.lower()}"
        if path_lower.exists():
            return path_lower
        path_upper = image_dir / f"{basename}{ext.upper()}"
        if path_upper.exists():
            return path_upper
    return None

def polygon_to_yolo_bbox(polygon_coords_norm):
    if len(polygon_coords_norm) < 2 or len(polygon_coords_norm) % 2 != 0:
        return None
    x_coords = np.array(polygon_coords_norm[0::2])
    y_coords = np.array(polygon_coords_norm[1::2])
    x_min, x_max = np.min(x_coords), np.max(x_coords)
    y_min, y_max = np.min(y_coords), np.max(y_coords)
    center_x = (x_min + x_max) / 2.0
    center_y = (y_min + y_max) / 2.0
    width = x_max - x_min
    height = y_max - y_min
    return center_x, center_y, width, height

# Prepare Directory Structure
print("Creating directory structure...")
train_img_dir = YOLO_DATA_DIR / "images" / "train"
val_img_dir = YOLO_DATA_DIR / "images" / "val"
train_label_dir = YOLO_DATA_DIR / "labels" / "train"
val_label_dir = YOLO_DATA_DIR / "labels" / "val"
shutil.rmtree(YOLO_DATA_DIR, ignore_errors=True)
train_img_dir.mkdir(parents=True, exist_ok=True)
val_img_dir.mkdir(parents=True, exist_ok=True)
train_label_dir.mkdir(parents=True, exist_ok=True)
val_label_dir.mkdir(parents=True, exist_ok=True)
print(f"Created: {YOLO_DATA_DIR}")

# Read Class Names
print(f"Reading class names from {CLASSES_FILE}...")
if not CLASSES_FILE.exists():
    raise FileNotFoundError(f"Error: classes.txt not found at {CLASSES_FILE}")
with open(CLASSES_FILE, 'r') as f:
    class_names = [line.strip() for line in f if line.strip()]
num_classes = len(class_names)
if num_classes == 0:
    raise ValueError(f"Error: No class names found in {CLASSES_FILE}")
print(f"Found {num_classes} classes: {class_names}")

# List & Split Data
print("Listing and splitting data...")
image_basenames = []
if not ORIG_IMAGE_DIR.is_dir() or not ORIG_LABEL_DIR.is_dir():
    raise FileNotFoundError(f"Error: Original image ({ORIG_IMAGE_DIR}) or label ({ORIG_LABEL_DIR}) directory not found.")
supported_img_formats = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp')
for label_file in ORIG_LABEL_DIR.glob("*.txt"):
    basename = label_file.stem
    found_image = False
    for ext in supported_img_formats:
        if (ORIG_IMAGE_DIR / f"{basename}{ext.lower()}").exists() or \
           (ORIG_IMAGE_DIR / f"{basename}{ext.upper()}").exists():
            image_basenames.append(basename)
            found_image = True
            break
if not image_basenames:
    raise ValueError(f"Error: No matching image/label pairs found in {ORIG_IMAGE_DIR} and {ORIG_LABEL_DIR}")
print(f"Found {len(image_basenames)} matching image/label pairs.")
random.shuffle(image_basenames)
split_index = int(len(image_basenames) * TRAIN_RATIO)
train_basenames = image_basenames[:split_index]
val_basenames = image_basenames[split_index:]
print(f"Splitting into {len(train_basenames)} train and {len(val_basenames)} validation samples.")

# Process Data
print("Processing training data...")
for basename in tqdm(train_basenames, desc="Train"):
    orig_img_path = find_image_path(ORIG_IMAGE_DIR, basename)
    orig_label_path = ORIG_LABEL_DIR / f"{basename}.txt"
    if orig_img_path and orig_label_path.exists():
        dest_img_path = train_img_dir / orig_img_path.name
        dest_label_path = train_label_dir / f"{basename}.txt"
        shutil.copy2(orig_img_path, dest_img_path)
        with open(orig_label_path, 'r') as infile, open(dest_label_path, 'w') as outfile:
            for line in infile:
                parts = line.strip().split()
                if len(parts) >= 3 and len(parts) % 2 != 0:
                    try:
                        class_id = int(parts[0])
                        polygon_coords_norm = [float(p) for p in parts[1:]]
                        yolo_bbox = polygon_to_yolo_bbox(polygon_coords_norm)
                        if yolo_bbox:
                            center_x, center_y, width, height = yolo_bbox
                            outfile.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
                    except ValueError:
                        continue

print("Processing validation data...")
for basename in tqdm(val_basenames, desc="Validation"):
    orig_img_path = find_image_path(ORIG_IMAGE_DIR, basename)
    orig_label_path = ORIG_LABEL_DIR / f"{basename}.txt"
    if orig_img_path and orig_label_path.exists():
        dest_img_path = val_img_dir / orig_img_path.name
        dest_label_path = val_label_dir / f"{basename}.txt"
        shutil.copy2(orig_img_path, dest_img_path)
        with open(orig_label_path, 'r') as infile, open(dest_label_path, 'w') as outfile:
            for line in infile:
                parts = line.strip().split()
                if len(parts) >= 3 and len(parts) % 2 != 0:
                    try:
                        class_id = int(parts[0])
                        polygon_coords_norm = [float(p) for p in parts[1:]]
                        yolo_bbox = polygon_to_yolo_bbox(polygon_coords_norm)
                        if yolo_bbox:
                            center_x, center_y, width, height = yolo_bbox
                            outfile.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
                    except ValueError:
                        continue
print("Data processing complete.")

# Create data.yaml
print("Creating data.yaml...")
data_yaml_path = YOLO_DATA_DIR / "data.yaml"
data_config = {
    'path': str(YOLO_DATA_DIR.resolve()),
    'train': str((YOLO_DATA_DIR / "images" / "train").resolve()),
    'val': str((YOLO_DATA_DIR / "images" / "val").resolve()),
    'nc': num_classes,
    'names': class_names
}
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False, sort_keys=False)
print(f"Created {data_yaml_path}")
print("--- data.yaml content ---")
with open(data_yaml_path, 'r') as f:
    print(f.read())
print("-------------------------")

# Train the Model
print("\nStarting YOLOv8 training...")
print(f"Using model: {MODEL_NAME}")
print(f"Epochs: {EPOCHS}, Image Size: {IMG_SIZE}, Batch Size: {BATCH_SIZE}")
print(f"Optimizer: AdamW, Initial LR: {INITIAL_LR}, Augmentation: Enabled")

model = YOLO(MODEL_NAME)

try:
    results = model.train(
        data=str(data_yaml_path),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        optimizer='AdamW',
        lr0=INITIAL_LR,
        lrf=FINAL_LR,
        augment=True,
        degrees=10,
        translate=0.1,
        scale=0.2,
        shear=0.2,
        project='YOLOv8_Training',
        name='my_custom_run_adamw_aug'
    )
    print("\nTraining finished!")
    print(f"Check for training results in: YOLOv8_Training/my_custom_run_adamw_aug")

except Exception as e:
    print(f"\nAn error occurred during training: {e}")
    print("Check GPU memory (try lowering batch size), file paths, and hyperparameters.")

Test trained ***model***

In [ ]:
import os
from pathlib import Path
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Configuration
MODEL_WEIGHTS_PATH = "/content/content/content/YOLOv8_Training/my_custom_run/weights/best.pt"
INPUT_PATH = "/content/content/content/test"
OUTPUT_DIR = "/content/inference_results"
CONF_THRESHOLD = 0.1

# Main Execution
model_path = Path(MODEL_WEIGHTS_PATH)
input_path = Path(INPUT_PATH)
output_dir = Path(OUTPUT_DIR)
supported_formats = ('.jpg', '.jpeg', '.png', '.bmp', '.webp', '.gif')

print("--- Script Start ---")
print(f"Model Path: {model_path}")
print(f"Input Path: {input_path}")
print(f"Output Dir: {output_dir}")

if not model_path.exists():
    print(f"Error: Model weights not found at {model_path}")
    exit()

if not input_path.exists():
    print(f"Error: Input path not found at {input_path}")
    exit()

output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output directory ensured: {output_dir}")

# Identify Image Files
print("\n--- Identifying Image Files ---")
print(f"Supported formats: {supported_formats}")

image_files = []
if input_path.is_file():
    if input_path.suffix.lower() in supported_formats:
        image_files.append(input_path)
        print(f"Added single file: {input_path.name}")
    else:
        print(f"Error: Input file {input_path} is not a supported image format.")
        exit()
elif input_path.is_dir():
    print(f"Input path is a directory. Scanning...")
    for item in input_path.iterdir():
        if item.is_file() and item.suffix.lower() in supported_formats:
            image_files.append(item)
    if not image_files:
        print(f"\nError: No supported image files found in directory: {input_path}")
        exit()
    print(f"Found {len(image_files)} supported image files.")
else:
    print(f"Error: Input path {input_path} is neither a file nor a directory.")
    exit()

print(f"Files to process: {[f.name for f in image_files]}")

# Load the Model
print("\n--- Loading Model ---")
try:
    model = YOLO(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Process Each Image
print(f"\n--- Starting Inference Process ({len(image_files)} images) ---")
for image_path in tqdm(image_files, desc="Processing Images"):
    output_filename = output_dir / f"{image_path.stem}.txt"
    try:
        results = model.predict(source=image_path, conf=CONF_THRESHOLD, verbose=False, stream=False)
        detections = []
        if results and results[0] and results[0].boxes:
            for box in results[0].boxes:
                class_id = int(box.cls.item())
                class_name = model.names[class_id]
                confidence = box.conf.item()
                xmin, ymin, xmax, ymax = map(float, box.xyxy.tolist()[0])
                detections.append(f"{class_name},{xmin:.2f},{ymin:.2f},{xmax:.2f},{ymax:.2f},{confidence:.4f}")

        with open(output_filename, 'w') as f:
            for line in detections:
                f.write(line + '\n')
        print(f"Processed: {image_path.name} -> Saved {len(detections)} detections to {output_filename.name}")

    except Exception as e:
        print(f"Error processing {image_path.name}: {e}")
        try:
            with open(output_filename, 'w') as f:
                f.write(f"Error processing image: {e}\n")
        except:
            print(f"Failed to write error message to {output_filename.name}")

print("\n--- Inference finished ---")
print(f"Results saved in: {output_dir}")